<a href="https://colab.research.google.com/github/claudiasfms/SAD/blob/master/3%C2%BATPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Cláudia Santos nº20160291
#entrega dia 19 Novembro
"""Crie um ficheiro em python para trabalhar o dataset datasets.california_housing
Nesse ficheiro, crie um script (função) por alínea que lhe permita gerar novos datasets a partir do dataset principal, onde tenha usado cada um dos seguintes métodos de pre-processamento:

1) Aggregation (houses_below.aggregate(["sum", "mean", "max"]))
2) Sampling  
4) Dimensionality Reduction 
5) Feature Subset Selection 
6) Feature Creation 
7) Discretization and Binarization 
8) Attribute Transformation """

#importar módulos
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from functools import reduce 

#importação do dataset
data = datasets.california_housing.fetch_california_housing()

print(data.DESCR)
print(data.feature_names)  #imprime os nomes principais (classes)
data.keys()  #vai buscar as chaves do dicionário

#mostra os dados em colunas (1ºs 5 (head))
dframe = pd.DataFrame(data.data, columns=data.feature_names)
dframe.head()

#mostra os dados mas desta vez com as classes
alldata = np.column_stack((data.data,data.target))
allnames = np.append(data.feature_names,"Classe")
dframe = pd.DataFrame(alldata, columns=allnames)
dframe.head()
dframe

stat = dframe.groupby("Classe").agg(["mean","count", "std"])
stat

#função p gerar novos datasets a partir do dataset principal
#1) Aggregation
houses_below = dframe.loc[dframe["MedInc"]<10,:]
houses_below[:10]
houses_below.count()
houses_below.aggregate(["sum", "mean", "max"])


#2) Sampling
random_subset = dframe.sample(n=3)
print(random_subset.head())


#4) Dimensionality Reduction e 5) Feature Subset Selection 
df=dframe.drop('Latitude', 1) 
df.corr()
df=dframe.drop(['AveBedrms', 'Longitude'], axis=1)  #remover colunas
model = RandomForestRegressor(random_state=1, max_depth=10)  #fazer um modelo com random forest
df=pd.get_dummies(df)  #converte variaveis categóricas em variáveis indicadoras
model.fit(df, dframe.Latitude)
#fazer o gráfico
features = df.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-5:]  #top 5 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


#6) Feature Creation
# É criado um vetor com o tamanho de "data", colocando número de ocorrências de cada letra na posição apropriada do vetor
dictionary = list(enumerate(set(list(reduce(lambda x, y: x + y, data)))))
def vectorize(text): 
    vector = np.zeros(len(dictionary)) 
    for i, word in dictionary: 
        num = 0 
        for w in text: 
            if w == word: 
                num += 1 
        if num: 
            vector[i] = num 
    return vector
for t in data: 
    print(vectorize(t))


#7) Discretization and Binarization 
plt.figure()
plt.grid()
plt.axis('equal')
data1 = np.random.randn(200,2)
data2 = np.random.randn(200,2)+1.75
plt.plot(data1[:,0],data1[:,1],  '.', color="b")
plt.plot(data2[:,0],data2[:,1],  '.', color="r")
plt.show()


#8) Attribute Transformation
#1a opção: merging
df.groupby('HouseAge')["Population"].sum()   #ordena os dados pela HouseAge
#vai combinar estas duas colunas
total = df.groupby('HouseAge')["Population"].sum().rename("Total").reset_index()
df_1 = df.merge(total)
df_1["Percentagem Total"] = df_1["Population"] / df_1["Total"]
df_1

#2a opção: transform
df.groupby('HouseAge')["Population"].transform('sum')
df["Total"] = df.groupby('HouseAge')["Population"].transform('sum')
df["Percentagem Total"] = df["Population"] / df["Total"]
df